# Преобразование файлов в формат $TeX$
На предыдущем этапе (*см. arXiv-API.ipynb*) посредством arXiv API были скачаны на диск исходники статей. Данные файлы не имееют расширения. Какие-то файлы заархивированы (*tar* или *gzip*), какие-то -- нет (*pdf* или *tex*). Файлы, которые не содержат точку в названии как правило являются *tex*-файлами (предположение). Если файл не является архивом и не содержит точку в названии, то он предположительно имеет формат *pdf* (такие файлы пропускались).  
  
[Здесь]() будет ссылка на архив с исходниками. 

Загрузка необходимых библиотек

In [5]:
import os
import gzip
import tarfile
import time

Указание пути к папке с исходниками (*path_to_sources*) и к папке, куда будут записываться преобразованные файлы (*path_to_tex*)

In [1]:
path_to_sources = 'C:\\Users\\user\\Downloads\\arxiv\\source\\'
path_to_tex = 'C:\\Users\\user\\Downloads\\arxiv\\source\\valid\\'

Реализация функции, которая преобразовывает файл в формат *tex* и сохраняет в указанную папку, а в исключитеном случаее (если файл не найден, файл уже есть в папке назначения или др.) ничего не делает (оператор pass).  

Если в архиве содержится несколько *tex*-файлов, то они объединяются в один документ.

In [175]:
def file_to_tex(filename, path_to_sources=path_to_sources, path_to_tex=path_to_tex):
    '''
    Description
    '''
    try:
        if '.' not in filename:
            os.rename(path_to_sources + filename, path_to_tex + filename + '.tex')
        
        else:
            try:
                tar = tarfile.open(path_to_sources + filename)
                names = tar.getnames()   
                names_tex = [name for name in names if '.tex' in name]
                if len(names_tex) > 1:                                      # если в архиве несколько tex-файлов
                    s = b''
                    for name in names_tex:
                        part = tar.extractfile(name).read()
                        s += part
                elif len(names_tex) == 1:
                    s = tar.extractfile(names_tex[0]).read()
                tar.close() 
                
                f = open(path_to_tex + filename + '.tex', 'wb')
                f.write(s)
                f.close()
                
            except tarfile.ReadError:
                fread = open(path_to_sources + filename,'rb').read()
            
                try:
                    s = gzip.decompress(fread)
                    fwrite = open(path_to_tex + filename + '.tex','wb')
                    fwrite.write(s)
                    fwrite.close()
                except OSError:
                    pass
    except Exception:
        pass

Выполним процедуру преобразования для имеющихся файлов

In [178]:
%%time

source_files = os.listdir(path_to_sources)
tex_files = os.listdir(path_to_tex)

counter = 0
for file in source_files:
    if file not in tex_files:
        counter += 1
        file_to_tex(file)
print('%d files was processed!' % counter)

2723 files was processed!
Wall time: 16min 22s


Скорость преобразования, как можно заметить, ~3 файл/c. Значит, для обработки 100K файлов потребуется ~9 часов. Эта оценка сильно зависит от того как много среди этих файлов архивов и от мощности компьюера.